In [ ]:
!pip install pycaret[full]

  Using cached matplotlib-3.7.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached shap-0.44.1-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (24 kB)
  Using cached interpret-0.7.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [61 lines of output]
      /var/tmp/pbs.11346558.pbs101/pip-build-env-am_8cpka/overlay/lib/python3.10/site-packages/setuptools/dist.py:759: SetuptoolsDeprecationWarning: License classifiers are deprecated.
      !!
      
              ********************************************************************************
              Please consider removing the following classifiers in favor of a SPDX license express

In [ ]:
!pip install skorch

In [ ]:
!pip install pytorch-ignite

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch import tensor

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import RidgeClassifier, LogisticRegression, SGDClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from catboost import CatBoostClassifier

import shap
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, root_mean_squared_error, mean_squared_log_error, root_mean_squared_log_error, mean_absolute_percentage_error, median_absolute_error, max_error

import torch.nn as nn

from skorch import NeuralNetClassifier, NeuralNetRegressor
from sklearn.pipeline import Pipeline
from skorch.helper import DataFrameTransformer
import skorch

from sklearn.model_selection import RandomizedSearchCV

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
df = pd.read_csv('mech_prop_training_dataset.csv')
df

/var/tmp/pbs.11346558.pbs101/ipykernel_29445/4254923679.py:1: DtypeWarning: Columns (31,43,65) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('For_mech_prop_predictions_13_July_2024.csv')


,material_id,K_VRH,G_VRH,Pugh_Ratio,builder_meta,nsites,elements,nelements,composition_reduced,formula_pretty,...,maximum local difference in GSmagmom,range local difference in GSmagmom,mean local difference in GSmagmom,avg_dev local difference in GSmagmom,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,range local difference in SpaceGroupNumber,mean local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,exists
0,mp-1,2.0,0.0,inf,emmet_version='0.72.20' pymatgen_version='2023...,1.0,[Element Cs],1.0,Cs1,Cs,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,True
1,mp-10,80.0,43.0,1.860465,emmet_version='0.72.20' pymatgen_version='2023...,2.0,[Element As],1.0,As1,As,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,True
2,mp-1000,26.0,16.0,1.625000,emmet_version='0.72.20' pymatgen_version='2023...,2.0,"[Element Ba, Element Te]",2.0,Ba1 Te1,BaTe,...,0.000000,0.000000e+00,0.000000,0.000000e+00,76.999987,76.999987,0.000000,76.999987,0.000000,True
3,mp-10000,131.0,74.0,1.770270,emmet_version='0.72.20' pymatgen_version='2023...,6.0,"[Element Hf, Element S]",2.0,Hf2 S1,Hf2S,...,0.000000,0.000000e+00,0.000000,0.000000e+00,44.415333,96.285363,51.870030,61.705343,23.053347,True
4,mp-10003,191.0,97.0,1.969072,emmet_version='0.72.20' pymatgen_version='2023...,12.0,"[Element Co, Element Nb, Element Si]",3.0,Nb4 Co1 Si1,Nb4CoSi,...,1.288343,1.288343e+00,0.396151,2.973975e-01,1.621693,29.120347,27.498654,9.467731,6.550872,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16498,mp-999550,132.0,48.0,2.750000,emmet_version='0.72.20' pymatgen_version='2023...,4.0,"[Element Ga, Element Mn]",2.0,Mn3 Ga1,Mn3Ga,...,0.000310,2.406508e-04,0.000155,7.753017e-05,34.273413,153.000000,118.726587,76.500000,38.250000,True
16499,mp-999558,205.0,89.0,2.303371,emmet_version='0.72.20' pymatgen_version='2023...,4.0,"[Element Mn, Element Ni, Element Si]",3.0,Mn2 Si1 Ni1,Mn2SiNi,...,0.595205,4.617801e-01,0.297697,1.487538e-01,5.344055,6.896037,1.551982,5.896037,0.500000,True
16500,mp-999576,232.0,115.0,2.017391,emmet_version='0.72.20' pymatgen_version='2023...,4.0,"[Element Mn, Element Ru, Element Si]",3.0,Mn2 Si1 Ru1,Mn2SiRu,...,0.000190,5.421011e-20,0.000190,2.710505e-20,9.032166,26.879954,17.847788,16.500000,6.401924,True
16501,mp-9996,12.0,10.0,1.200000,emmet_version='0.72.20' pymatgen_version='2023...,24.0,"[Element P, Element Si]",2.0,Si1 P2,SiP2,...,0.000000,0.000000e+00,0.000000,0.000000e+00,37.463038,186.001437,148.538399,112.163384,49.582687,True


In [ ]:
# compositional attributes only
compositional_features_only_df = df[['0-norm',
 '2-norm',
 '3-norm',
 '5-norm',
 '7-norm',
 '10-norm',
 'MagpieData minimum Number',
 'MagpieData maximum Number',
 'MagpieData range Number',
 'MagpieData mean Number',
 'MagpieData avg_dev Number',
 'MagpieData mode Number',
 'MagpieData minimum MendeleevNumber',
 'MagpieData maximum MendeleevNumber',
 'MagpieData range MendeleevNumber',
 'MagpieData mean MendeleevNumber',
 'MagpieData avg_dev MendeleevNumber',
 'MagpieData mode MendeleevNumber',
 'MagpieData minimum AtomicWeight',
 'MagpieData maximum AtomicWeight',
 'MagpieData range AtomicWeight',
 'MagpieData mean AtomicWeight',
 'MagpieData avg_dev AtomicWeight',
 'MagpieData mode AtomicWeight',
 'MagpieData minimum MeltingT',
 'MagpieData maximum MeltingT',
 'MagpieData range MeltingT',
 'MagpieData mean MeltingT',
 'MagpieData avg_dev MeltingT',
 'MagpieData mode MeltingT',
 'MagpieData minimum Column',
 'MagpieData maximum Column',
 'MagpieData range Column',
 'MagpieData mean Column',
 'MagpieData avg_dev Column',
 'MagpieData mode Column',
 'MagpieData minimum Row',
 'MagpieData maximum Row',
 'MagpieData range Row',
 'MagpieData mean Row',
 'MagpieData avg_dev Row',
 'MagpieData mode Row',
 'MagpieData minimum CovalentRadius',
 'MagpieData maximum CovalentRadius',
 'MagpieData range CovalentRadius',
 'MagpieData mean CovalentRadius',
 'MagpieData avg_dev CovalentRadius',
 'MagpieData mode CovalentRadius',
 'MagpieData minimum Electronegativity',
 'MagpieData maximum Electronegativity',
 'MagpieData range Electronegativity',
 'MagpieData mean Electronegativity',
 'MagpieData avg_dev Electronegativity',
 'MagpieData mode Electronegativity',
 'MagpieData minimum NsValence',
 'MagpieData maximum NsValence',
 'MagpieData range NsValence',
 'MagpieData mean NsValence',
 'MagpieData avg_dev NsValence',
 'MagpieData mode NsValence',
 'MagpieData minimum NpValence',
 'MagpieData maximum NpValence',
 'MagpieData range NpValence',
 'MagpieData mean NpValence',
 'MagpieData avg_dev NpValence',
 'MagpieData mode NpValence',
 'MagpieData minimum NdValence',
 'MagpieData maximum NdValence',
 'MagpieData range NdValence',
 'MagpieData mean NdValence',
 'MagpieData avg_dev NdValence',
 'MagpieData mode NdValence',
 'MagpieData minimum NfValence',
 'MagpieData maximum NfValence',
 'MagpieData range NfValence',
 'MagpieData mean NfValence',
 'MagpieData avg_dev NfValence',
 'MagpieData mode NfValence',
 'MagpieData minimum NValence',
 'MagpieData maximum NValence',
 'MagpieData range NValence',
 'MagpieData mean NValence',
 'MagpieData avg_dev NValence',
 'MagpieData mode NValence',
 'MagpieData minimum NsUnfilled',
 'MagpieData maximum NsUnfilled',
 'MagpieData range NsUnfilled',
 'MagpieData mean NsUnfilled',
 'MagpieData avg_dev NsUnfilled',
 'MagpieData mode NsUnfilled',
 'MagpieData minimum NpUnfilled',
 'MagpieData maximum NpUnfilled',
 'MagpieData range NpUnfilled',
 'MagpieData mean NpUnfilled',
 'MagpieData avg_dev NpUnfilled',
 'MagpieData mode NpUnfilled',
 'MagpieData minimum NdUnfilled',
 'MagpieData maximum NdUnfilled',
 'MagpieData range NdUnfilled',
 'MagpieData mean NdUnfilled',
 'MagpieData avg_dev NdUnfilled',
 'MagpieData mode NdUnfilled',
 'MagpieData minimum NfUnfilled',
 'MagpieData maximum NfUnfilled',
 'MagpieData range NfUnfilled',
 'MagpieData mean NfUnfilled',
 'MagpieData avg_dev NfUnfilled',
 'MagpieData mode NfUnfilled',
 'MagpieData minimum NUnfilled',
 'MagpieData maximum NUnfilled',
 'MagpieData range NUnfilled',
 'MagpieData mean NUnfilled',
 'MagpieData avg_dev NUnfilled',
 'MagpieData mode NUnfilled',
 'MagpieData minimum GSvolume_pa',
 'MagpieData maximum GSvolume_pa',
 'MagpieData range GSvolume_pa',
 'MagpieData mean GSvolume_pa',
 'MagpieData avg_dev GSvolume_pa',
 'MagpieData mode GSvolume_pa',
 'MagpieData minimum GSbandgap',
 'MagpieData maximum GSbandgap',
 'MagpieData range GSbandgap',
 'MagpieData mean GSbandgap',
 'MagpieData avg_dev GSbandgap',
 'MagpieData mode GSbandgap',
 'MagpieData minimum GSmagmom',
 'MagpieData maximum GSmagmom',
 'MagpieData range GSmagmom',
 'MagpieData mean GSmagmom',
 'MagpieData avg_dev GSmagmom',
 'MagpieData mode GSmagmom',
 'MagpieData minimum SpaceGroupNumber',
 'MagpieData maximum SpaceGroupNumber',
 'MagpieData range SpaceGroupNumber',
 'MagpieData mean SpaceGroupNumber',
 'MagpieData avg_dev SpaceGroupNumber',
 'MagpieData mode SpaceGroupNumber',
 'frac s valence electrons',
 'frac p valence electrons',
 'frac d valence electrons',
 'frac f valence electrons',
 'compound possible',
 'max ionic char',
 'avg ionic char', 'K_VRH', 'G_VRH', 'Pugh_Ratio']]
compositional_features_only_df

,0-norm,2-norm,3-norm,5-norm,7-norm,10-norm,MagpieData minimum Number,MagpieData maximum Number,MagpieData range Number,MagpieData mean Number,...,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,compound possible,max ionic char,avg ionic char,K_VRH,G_VRH,Pugh_Ratio
0,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,55.0,55.0,0.0,55.000000,...,1.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,2.0,0.0,inf
1,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,33.0,33.0,0.0,33.000000,...,0.133333,0.200000,0.666667,0.000000,1.0,0.000000,0.000000,80.0,43.0,1.860465
2,2.0,0.707107,0.629961,0.574349,0.552045,0.535887,52.0,56.0,4.0,54.000000,...,0.222222,0.222222,0.555556,0.000000,1.0,0.306515,0.076629,26.0,16.0,1.625000
3,2.0,0.745356,0.693361,0.670782,0.667408,0.666732,16.0,72.0,56.0,53.333333,...,0.142857,0.095238,0.095238,0.666667,0.0,0.336084,0.074685,131.0,74.0,1.770270
4,3.0,0.707107,0.673540,0.666927,0.666678,0.666667,14.0,41.0,27.0,34.166667,...,0.242424,0.060606,0.696970,0.000000,0.0,0.022249,0.004631,191.0,97.0,1.969072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16498,2.0,0.790569,0.759147,0.750616,0.750049,0.750001,25.0,31.0,6.0,26.500000,...,0.235294,0.029412,0.735294,0.000000,0.0,0.016758,0.003142,132.0,48.0,2.750000
16499,3.0,0.612372,0.538609,0.506099,0.501109,0.500098,14.0,28.0,14.0,23.000000,...,0.285714,0.071429,0.642857,0.000000,0.0,0.031881,0.007757,205.0,89.0,2.303371
16500,3.0,0.612372,0.538609,0.506099,0.501109,0.500098,14.0,44.0,30.0,27.000000,...,0.269231,0.076923,0.653846,0.000000,0.0,0.100238,0.017690,232.0,115.0,2.017391
16501,2.0,0.745356,0.693361,0.670782,0.667408,0.666732,14.0,15.0,1.0,14.666667,...,0.428571,0.571429,0.000000,0.000000,0.0,0.020806,0.004623,12.0,10.0,1.200000


In [ ]:
X_comp_features = compositional_features_only_df.drop(['K_VRH', 'G_VRH', 'Pugh_Ratio'], axis = 1)
y_comp_features = compositional_features_only_df[['K_VRH', 'G_VRH',	'Pugh_Ratio']]

In [ ]:
compositional_features_only_df[compositional_features_only_df.columns[0:100]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16503 entries, 0 to 16502
Data columns (total 100 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   0-norm                                16503 non-null  float64
 1   2-norm                                16503 non-null  float64
 2   3-norm                                16503 non-null  float64
 3   5-norm                                16503 non-null  float64
 4   7-norm                                16503 non-null  float64
 5   10-norm                               16503 non-null  float64
 6   MagpieData minimum Number             16503 non-null  float64
 7   MagpieData maximum Number             16503 non-null  float64
 8   MagpieData range Number               16503 non-null  float64
 9   MagpieData mean Number                16503 non-null  float64
 10  MagpieData avg_dev Number             16503 non-null  float64
 11  MagpieData mod

In [ ]:
compositional_features_only_df[compositional_features_only_df.columns[100:]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16503 entries, 0 to 16502
Data columns (total 48 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   MagpieData avg_dev NdUnfilled        16503 non-null  float64
 1   MagpieData mode NdUnfilled           16503 non-null  float64
 2   MagpieData minimum NfUnfilled        16503 non-null  float64
 3   MagpieData maximum NfUnfilled        16503 non-null  float64
 4   MagpieData range NfUnfilled          16503 non-null  float64
 5   MagpieData mean NfUnfilled           16503 non-null  float64
 6   MagpieData avg_dev NfUnfilled        16503 non-null  float64
 7   MagpieData mode NfUnfilled           16503 non-null  float64
 8   MagpieData minimum NUnfilled         16503 non-null  float64
 9   MagpieData maximum NUnfilled         16503 non-null  float64
 10  MagpieData range NUnfilled           16503 non-null  float64
 11  MagpieData mean NUnfilled   

In [ ]:
df.columns[0:100]

Index(['material_id', 'K_VRH', 'G_VRH', 'Pugh_Ratio', 'builder_meta', 'nsites',
       'elements', 'nelements', 'composition_reduced', 'formula_pretty',
       'formula_anonymous', 'chemsys', 'volume', 'density', 'density_atomic',
       'symmetry', 'property_name', 'deprecated', 'deprecation_reasons',
       'last_updated', 'origins', 'warnings', 'task_ids',
       'uncorrected_energy_per_atom', 'energy_per_atom',
       'formation_energy_per_atom', 'energy_above_hull', 'is_stable',
       'equilibrium_reaction_energy_per_atom', 'decomposes_to', 'xas',
       'grain_boundaries', 'band_gap', 'cbm', 'vbm', 'efermi', 'is_gap_direct',
       'is_metal', 'es_source_calc_id', 'bandstructure', 'dos',
       'dos_energy_up', 'dos_energy_down', 'is_magnetic', 'ordering',
       'total_magnetization', 'total_magnetization_normalized_vol',
       'total_magnetization_normalized_formula_units', 'num_magnetic_sites',
       'num_unique_magnetic_sites', 'types_of_magnetic_species',
       'bulk_mod

In [ ]:
df.columns[100:]

Index(['Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 'Ga', 'Ge', 'As',
       ...
       'maximum local difference in GSmagmom',
       'range local difference in GSmagmom',
       'mean local difference in GSmagmom',
       'avg_dev local difference in GSmagmom',
       'minimum local difference in SpaceGroupNumber',
       'maximum local difference in SpaceGroupNumber',
       'range local difference in SpaceGroupNumber',
       'mean local difference in SpaceGroupNumber',
       'avg_dev local difference in SpaceGroupNumber', 'exists'],
      dtype='object', length=361)

In [ ]:
import torch.nn as nn

from skorch import NeuralNetClassifier, NeuralNetRegressor
from sklearn.pipeline import Pipeline
from skorch.helper import DataFrameTransformer

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16503 entries, 0 to 16502
Columns: 461 entries, material_id to exists
dtypes: bool(7), float64(424), object(30)
memory usage: 57.3+ MB


In [ ]:
print("There are {} columns of names listed below:\n\n{}".format(df.shape[1], df.columns.values))

There are 461 columns of names listed below:

['material_id' 'K_VRH' 'G_VRH' 'Pugh_Ratio' 'builder_meta' 'nsites'
 'elements' 'nelements' 'composition_reduced' 'formula_pretty'
 'formula_anonymous' 'chemsys' 'volume' 'density' 'density_atomic'
 'symmetry' 'property_name' 'deprecated' 'deprecation_reasons'
 'last_updated' 'origins' 'warnings' 'task_ids'
 'uncorrected_energy_per_atom' 'energy_per_atom'
 'formation_energy_per_atom' 'energy_above_hull' 'is_stable'
 'equilibrium_reaction_energy_per_atom' 'decomposes_to' 'xas'
 'grain_boundaries' 'band_gap' 'cbm' 'vbm' 'efermi' 'is_gap_direct'
 'is_metal' 'es_source_calc_id' 'bandstructure' 'dos' 'dos_energy_up'
 'dos_energy_down' 'is_magnetic' 'ordering' 'total_magnetization'
 'total_magnetization_normalized_vol'
 'total_magnetization_normalized_formula_units' 'num_magnetic_sites'
 'num_unique_magnetic_sites' 'types_of_magnetic_species' 'bulk_modulus'
 'shear_modulus' 'universal_anisotropy' 'homogeneous_poisson' 'e_total'
 'e_ionic' 'e_elec

In [ ]:
df3 = pd.read_csv('mech_prop_validation_dataset.csv')
df3

,material_id,builder_meta,nsites,elements,nelements,composition_reduced,formula_pretty,formula_anonymous,chemsys,volume,...,avg_dev local difference in GSmagmom,minimum local difference in SpaceGroupNumber,maximum local difference in SpaceGroupNumber,range local difference in SpaceGroupNumber,mean local difference in SpaceGroupNumber,avg_dev local difference in SpaceGroupNumber,K_VRH,G_VRH,Pugh_Ratio,exists
0,mp-3949,emmet_version='0.72.20' pymatgen_version='2023...,64,"[Element K, Element Li, Element Si]",3,K7 Li1 Si8,K7LiSi8,AB7C8,K-Li-Si,1951.965448,...,0.0,1.049271,1.424875,0.375604,1.266754,0.135871,NaN,NaN,NaN,False
1,mp-4657,emmet_version='0.72.20' pymatgen_version='2023...,80,"[Element Ga, Element Li, Element O]",3,Li5 Ga1 O4,Li5GaO4,AB4C5,Ga-Li-O,748.454369,...,0.0,86.530203,191.228152,104.697949,151.220931,27.036697,NaN,NaN,NaN,False
2,mp-4995,emmet_version='0.72.20' pymatgen_version='2023...,20,"[Element Li, Element O, Element Sb]",3,Li1 Sb1 O3,LiSbO3,ABC3,Li-O-Sb,215.500133,...,0.0,100.642918,205.344802,104.701884,134.920881,35.801216,NaN,NaN,NaN,False
3,mp-5368,emmet_version='0.72.20' pymatgen_version='2023...,27,"[Element F, Element Ge, Element Li]",3,Li2 Ge1 F6,Li2GeF6,AB2C6,F-Ge-Li,272.775032,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,mp-5488,emmet_version='0.72.20' pymatgen_version='2023...,8,"[Element In, Element Li, Element O]",3,Li1 In1 O2,LiInO2,ABC2,In-Li-O,87.122210,...,0.0,126.862161,204.842529,77.980367,166.352988,19.745413,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,mp-2712854,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element Ge, Element O, Element P,...",5,Al1 Ge1 P1 S2 O12,AlGeP(SO6)2,ABCD2E12,Al-Ge-O-P-S,1283.361072,...,0.0,10.000000,213.000000,203.000000,52.359925,39.124741,NaN,NaN,NaN,False
4739,mp-2712928,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element O, Element P, Element S, ...",5,Al1 Sn1 P1 S2 O12,AlSnP(SO6)2,ABCD2E12,Al-O-P-S-Sn,1394.547188,...,0.0,10.000000,213.000000,203.000000,44.558663,32.913571,NaN,NaN,NaN,False
4740,mp-2713730,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element Ga, Element O, Element S]",4,Al1 Ga1 S3 O12,AlGa(SO4)3,ABC3D12,Al-Ga-O-S,1307.718619,...,0.0,13.475931,213.000000,199.524069,41.671618,27.134342,NaN,NaN,NaN,False
4741,mp-2713752,emmet_version='0.72.20' pymatgen_version='2023...,102,"[Element Al, Element O, Element S]",3,Al2 S3 O12,Al2(SO4)3,A2B3C12,Al-O-S,1272.270712,...,0.0,19.363213,213.000000,193.636787,56.762943,37.198269,NaN,NaN,NaN,False


In [ ]:
len(X_comp_features.columns)

145

In [ ]:
torch.manual_seed(0)

In [ ]:
from torch import nn
import torch.nn.functional as F

class Regressor(nn.Module):
    def __init__(self, num_inputs=len(X_comp_features.columns), num_units_d1=300, num_units_d2=100, dropout = 0.4):
        super(Regressor, self).__init__()

        self.first_layer = nn.Linear(num_inputs, num_units_d1)
        self.batch_norm1 = nn.BatchNorm1d(num_units_d1)
        self.dropout = nn.Dropout(dropout)
        self.second_layer = nn.Linear(num_units_d1,num_units_d2)
        self.batch_norm2 = nn.BatchNorm1d(num_units_d2)
        self.final_layer = nn.Linear(num_units_d2,1)

    def forward(self, x_batch):
        X = self.first_layer(x_batch)
        X = self.batch_norm1(X)
        X = F.relu(X)
        X= self.dropout(X)
        X = self.second_layer(X)
        X = self.batch_norm2(X)
        X = F.relu(X)

        return self.final_layer(X)

In [ ]:
from skorch import NeuralNetRegressor
from torch import optim

skorch_regressor = NeuralNetRegressor(module=Regressor, optimizer=optim.RMSprop, max_epochs=200, verbose=0, lr = 0.0005, train_split=None, batch_size = 128)

skorch_regressor

<class 'skorch.regressor.NeuralNetRegressor'>[uninitialized](
  module=<class '__main__.Regressor'>,
)

In [ ]:
skorch_regressor.initialize().load_params(
                              f_params="params.pkl",
                              f_optimizer="opt.pkl",
                              f_criterion="criterion.pkl",
                              f_history="hist.json"
                            )

In [ ]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X_comp_features, y_comp_features['G_VRH'], train_size=0.8, random_state=0)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_holdout = scaler.transform(X_holdout)

In [ ]:
from torch import tensor
X_train = tensor(X_train, dtype=torch.float32)
X_holdout = tensor(X_holdout, dtype=torch.float32)
y_train = tensor(y_train.values.reshape(-1,1), dtype=torch.float32)
y_holdout = tensor(y_holdout.values.reshape(-1,1), dtype=torch.float32)

X_train.shape, X_holdout.shape, y_train.shape, y_holdout.shape

(torch.Size([13202, 145]),
 torch.Size([3301, 145]),
 torch.Size([13202, 1]),
 torch.Size([3301, 1]))

In [ ]:
skorch_regressor.fit(X_train, y_train)

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Regressor(
    (first_layer): Linear(in_features=145, out_features=300, bias=True)
    (batch_norm1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.4, inplace=False)
    (second_layer): Linear(in_features=300, out_features=100, bias=True)
    (batch_norm2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (final_layer): Linear(in_features=100, out_features=1, bias=True)
  ),
)

In [ ]:
y_holdout_preds = skorch_regressor.predict(X_holdout)

In [ ]:
type(y_holdout_preds)

numpy.ndarray

In [ ]:
y_holdout_preds.shape

(3301, 1)

In [ ]:
print("Train MSE : {}".format(mean_squared_error(y_train, skorch_regressor.predict(X_train).reshape(-1))))
print("Holdout  MSE : {}".format(mean_squared_error(y_holdout, skorch_regressor.predict(X_holdout).reshape(-1))))

print("\nTrain R^2 : {}".format(skorch_regressor.score(X_train, y_train)))
print("Holdout  R^2 : {}".format(skorch_regressor.score(X_holdout, y_holdout)))

Train MSE : 261.2062072753906
Holdout  MSE : 414.1552429199219

Train R^2 : 0.812240838437528
Holdout  R^2 : 0.7005837471590532


In [ ]:
skorch_regressor.get_params()

{'module': __main__.Regressor,
 'criterion': torch.nn.modules.loss.MSELoss,
 'optimizer': torch.optim.rmsprop.RMSprop,
 'lr': 0.0005,
 'max_epochs': 200,
 'batch_size': 128,
 'iterator_train': torch.utils.data.dataloader.DataLoader,
 'iterator_valid': torch.utils.data.dataloader.DataLoader,
 'dataset': skorch.dataset.Dataset,
 'train_split': None,
 'callbacks': None,
 'predict_nonlinearity': 'auto',
 'warm_start': False,
 'verbose': 0,
 'device': 'cpu',
 'compile': False,
 'use_caching': 'auto',
 '_params_to_validate': set(),
 'callbacks__epoch_timer': <skorch.callbacks.logging.EpochTimer at 0x1532186bf3d0>,
 'callbacks__train_loss': <skorch.callbacks.scoring.PassthroughScoring at 0x1532186bf2b0>,
 'callbacks__train_loss__name': 'train_loss',
 'callbacks__train_loss__lower_is_better': True,
 'callbacks__train_loss__on_train': True,
 'callbacks__valid_loss': <skorch.callbacks.scoring.PassthroughScoring at 0x1532186bf310>,
 'callbacks__valid_loss__name': 'valid_loss',
 'callbacks__valid_

In [ ]:
skorch_regressor

<class 'skorch.regressor.NeuralNetRegressor'>[initialized](
  module_=Regressor(
    (first_layer): Linear(in_features=145, out_features=300, bias=True)
    (batch_norm1): BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (dropout): Dropout(p=0.4, inplace=False)
    (second_layer): Linear(in_features=300, out_features=100, bias=True)
    (batch_norm2): BatchNorm1d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (final_layer): Linear(in_features=100, out_features=1, bias=True)
  ),
)

In [ ]:
TRAIN_PRED = pd.DataFrame()
HOLDOUT_PRED = pd.DataFrame()

NEW_TEST_PRED_2 = pd.DataFrame()
NEW_TEST_PROBA_2 = pd.DataFrame()

TRAIN_R2 = pd.DataFrame()
TRAIN_MAE = pd.DataFrame()
TRAIN_MSE = pd.DataFrame()
TRAIN_RMSE = pd.DataFrame()
TRAIN_MSLE = pd.DataFrame()
TRAIN_RMSLE = pd.DataFrame()
TRAIN_MAPE = pd.DataFrame()
TRAIN_MEDAE = pd.DataFrame()
TRAIN_MAXE = pd.DataFrame()

HOLDOUT_R2 = pd.DataFrame()
HOLDOUT_MAE = pd.DataFrame()
HOLDOUT_MSE = pd.DataFrame()
HOLDOUT_RMSE = pd.DataFrame()
HOLDOUT_MSLE = pd.DataFrame()
HOLDOUT_RMSLE = pd.DataFrame()
HOLDOUT_MAPE = pd.DataFrame()
HOLDOUT_MEDAE = pd.DataFrame()
HOLDOUT_MAXE = pd.DataFrame()

In [ ]:
models= [skorch_regressor, skorch_regressor, skorch_regressor]

models_name = ['skorch_model_1', 'skorch_model_2', 'skorch_model_3']

In [ ]:
import time

In [ ]:
from sklearn.metrics import PredictionErrorDisplay

for reg_n, reg in zip(models_name,models):

    TRAIN_R2_result = []
    TRAIN_MAE_result = []
    TRAIN_MSE_result = []
    TRAIN_RMSE_result = []

    TRAIN_MAPE_result = []
    TRAIN_MEDAE_result = []
    TRAIN_MAXE_result = []

    HOLDOUT_R2_result = []
    HOLDOUT_MAE_result = []
    HOLDOUT_MSE_result = []
    HOLDOUT_RMSE_result = []

    HOLDOUT_MAPE_result = []
    HOLDOUT_MEDAE_result = []
    HOLDOUT_MAXE_result = []


    for i in range(0,100):
        print(time.ctime())
        # Train-test split of 80:20 ratio to produce train and holdout data
        X_train, X_holdout, y_train, y_holdout = train_test_split(X_comp_features, y_comp_features['G_VRH'], train_size=0.8, random_state=i)

        scaler = StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)
        X_holdout = scaler.transform(X_holdout)

        X_train = tensor(X_train, dtype=torch.float32)
        X_holdout = tensor(X_holdout, dtype=torch.float32)
        y_train = tensor(y_train.values.reshape(-1,1), dtype=torch.float32)
        y_holdout = tensor(y_holdout.values.reshape(-1,1), dtype=torch.float32)

        # compositional attributes only
        NEW_TEST_DB2 = df3[['0-norm',
                          '2-norm',
                          '3-norm',
                          '5-norm',
                          '7-norm',
                          '10-norm',
                          'MagpieData minimum Number',
                          'MagpieData maximum Number',
                          'MagpieData range Number',
                          'MagpieData mean Number',
                          'MagpieData avg_dev Number',
                          'MagpieData mode Number',
                          'MagpieData minimum MendeleevNumber',
                          'MagpieData maximum MendeleevNumber',
                          'MagpieData range MendeleevNumber',
                          'MagpieData mean MendeleevNumber',
                          'MagpieData avg_dev MendeleevNumber',
                          'MagpieData mode MendeleevNumber',
                          'MagpieData minimum AtomicWeight',
                          'MagpieData maximum AtomicWeight',
                          'MagpieData range AtomicWeight',
                          'MagpieData mean AtomicWeight',
                          'MagpieData avg_dev AtomicWeight',
                          'MagpieData mode AtomicWeight',
                          'MagpieData minimum MeltingT',
                          'MagpieData maximum MeltingT',
                          'MagpieData range MeltingT',
                          'MagpieData mean MeltingT',
                          'MagpieData avg_dev MeltingT',
                          'MagpieData mode MeltingT',
                          'MagpieData minimum Column',
                          'MagpieData maximum Column',
                          'MagpieData range Column',
                          'MagpieData mean Column',
                          'MagpieData avg_dev Column',
                          'MagpieData mode Column',
                          'MagpieData minimum Row',
                          'MagpieData maximum Row',
                          'MagpieData range Row',
                          'MagpieData mean Row',
                          'MagpieData avg_dev Row',
                          'MagpieData mode Row',
                          'MagpieData minimum CovalentRadius',
                          'MagpieData maximum CovalentRadius',
                          'MagpieData range CovalentRadius',
                          'MagpieData mean CovalentRadius',
                          'MagpieData avg_dev CovalentRadius',
                          'MagpieData mode CovalentRadius',
                          'MagpieData minimum Electronegativity',
                          'MagpieData maximum Electronegativity',
                          'MagpieData range Electronegativity',
                          'MagpieData mean Electronegativity',
                          'MagpieData avg_dev Electronegativity',
                          'MagpieData mode Electronegativity',
                          'MagpieData minimum NsValence',
                          'MagpieData maximum NsValence',
                          'MagpieData range NsValence',
                          'MagpieData mean NsValence',
                          'MagpieData avg_dev NsValence',
                          'MagpieData mode NsValence',
                          'MagpieData minimum NpValence',
                          'MagpieData maximum NpValence',
                          'MagpieData range NpValence',
                          'MagpieData mean NpValence',
                          'MagpieData avg_dev NpValence',
                          'MagpieData mode NpValence',
                          'MagpieData minimum NdValence',
                          'MagpieData maximum NdValence',
                          'MagpieData range NdValence',
                          'MagpieData mean NdValence',
                          'MagpieData avg_dev NdValence',
                          'MagpieData mode NdValence',
                          'MagpieData minimum NfValence',
                          'MagpieData maximum NfValence',
                          'MagpieData range NfValence',
                          'MagpieData mean NfValence',
                          'MagpieData avg_dev NfValence',
                          'MagpieData mode NfValence',
                          'MagpieData minimum NValence',
                          'MagpieData maximum NValence',
                          'MagpieData range NValence',
                          'MagpieData mean NValence',
                          'MagpieData avg_dev NValence',
                          'MagpieData mode NValence',
                          'MagpieData minimum NsUnfilled',
                          'MagpieData maximum NsUnfilled',
                          'MagpieData range NsUnfilled',
                          'MagpieData mean NsUnfilled',
                          'MagpieData avg_dev NsUnfilled',
                          'MagpieData mode NsUnfilled',
                          'MagpieData minimum NpUnfilled',
                          'MagpieData maximum NpUnfilled',
                          'MagpieData range NpUnfilled',
                          'MagpieData mean NpUnfilled',
                          'MagpieData avg_dev NpUnfilled',
                          'MagpieData mode NpUnfilled',
                          'MagpieData minimum NdUnfilled',
                          'MagpieData maximum NdUnfilled',
                          'MagpieData range NdUnfilled',
                          'MagpieData mean NdUnfilled',
                          'MagpieData avg_dev NdUnfilled',
                          'MagpieData mode NdUnfilled',
                          'MagpieData minimum NfUnfilled',
                          'MagpieData maximum NfUnfilled',
                          'MagpieData range NfUnfilled',
                          'MagpieData mean NfUnfilled',
                          'MagpieData avg_dev NfUnfilled',
                          'MagpieData mode NfUnfilled',
                          'MagpieData minimum NUnfilled',
                          'MagpieData maximum NUnfilled',
                          'MagpieData range NUnfilled',
                          'MagpieData mean NUnfilled',
                          'MagpieData avg_dev NUnfilled',
                          'MagpieData mode NUnfilled',
                          'MagpieData minimum GSvolume_pa',
                          'MagpieData maximum GSvolume_pa',
                          'MagpieData range GSvolume_pa',
                          'MagpieData mean GSvolume_pa',
                          'MagpieData avg_dev GSvolume_pa',
                          'MagpieData mode GSvolume_pa',
                          'MagpieData minimum GSbandgap',
                          'MagpieData maximum GSbandgap',
                          'MagpieData range GSbandgap',
                          'MagpieData mean GSbandgap',
                          'MagpieData avg_dev GSbandgap',
                          'MagpieData mode GSbandgap',
                          'MagpieData minimum GSmagmom',
                          'MagpieData maximum GSmagmom',
                          'MagpieData range GSmagmom',
                          'MagpieData mean GSmagmom',
                          'MagpieData avg_dev GSmagmom',
                          'MagpieData mode GSmagmom',
                          'MagpieData minimum SpaceGroupNumber',
                          'MagpieData maximum SpaceGroupNumber',
                          'MagpieData range SpaceGroupNumber',
                          'MagpieData mean SpaceGroupNumber',
                          'MagpieData avg_dev SpaceGroupNumber',
                          'MagpieData mode SpaceGroupNumber',
                          'frac s valence electrons',
                          'frac p valence electrons',
                          'frac d valence electrons',
                          'frac f valence electrons',
                          'compound possible',
                          'max ionic char',
                          'avg ionic char']]
        NEW_TEST_DB2 = scaler.transform(NEW_TEST_DB2)
        NEW_TEST_DB2 = tensor(NEW_TEST_DB2, dtype=torch.float32)


        # Train the model and perform predictions
        reg.fit(X_train,y_train)
        y_train_pred = reg.predict(X_train)
        y_holdout_pred = reg.predict(X_holdout)

        X_train = pd.DataFrame(X_train.numpy())
        X_holdout = pd.DataFrame(X_holdout.numpy())
        y_train = pd.DataFrame(y_train.numpy())
        y_holdout = pd.DataFrame(y_holdout.numpy())

        new_test_pred_2 = reg.predict(NEW_TEST_DB2)

        # Append to dataframe the prediced target labels (Train, holdout, and test) made by the models
        Train_Pred = pd.DataFrame(y_train_pred, columns=[str(reg_n)+'_'+str(i)])
        Holdout_Pred = pd.DataFrame(y_holdout_pred, columns=[str(reg_n)+'_'+str(i)])
        New_Test_pred_2 = pd.DataFrame(new_test_pred_2, columns=[str(reg_n)+'_'+str(i)])

        # Training performance metrics scores
        Train_r2 = r2_score(y_train, y_train_pred)
        Train_mae = mean_absolute_error(y_train, y_train_pred)
        Train_mse = mean_squared_error(y_train, y_train_pred)
        Train_rmse = root_mean_squared_error(y_train, y_train_pred)

        Train_mape = mean_absolute_percentage_error(y_train, y_train_pred)
        Train_medae = median_absolute_error(y_train, y_train_pred)
        Train_maxe = max_error(y_train, y_train_pred)

        # Holdout performance metrics scores
        Holdout_r2 = r2_score(y_holdout, y_holdout_pred)
        Holdout_mae = mean_absolute_error(y_holdout, y_holdout_pred)
        Holdout_mse = mean_squared_error(y_holdout, y_holdout_pred)
        Holdout_rmse = root_mean_squared_error(y_holdout, y_holdout_pred)

        Holdout_mape = mean_absolute_percentage_error(y_holdout, y_holdout_pred)
        Holdout_medae = median_absolute_error(y_holdout, y_holdout_pred)
        Holdout_maxe = max_error(y_holdout, y_holdout_pred)

        # Append training performance metrics scores of each model to dataframes
        TRAIN_R2_result.append(Train_r2)
        TRAIN_MAE_result.append(Train_mae)
        TRAIN_MSE_result.append(Train_mse)
        TRAIN_RMSE_result.append(Train_rmse)
        TRAIN_MAPE_result.append(Train_mape)
        TRAIN_MEDAE_result.append(Train_medae)
        TRAIN_MAXE_result.append(Train_maxe)

        # Append holdout performance metrics scores of each model to dataframes
        HOLDOUT_R2_result.append(Holdout_r2)
        HOLDOUT_MAE_result.append(Holdout_mae)
        HOLDOUT_MSE_result.append(Holdout_mse)
        HOLDOUT_RMSE_result.append(Holdout_rmse)
        HOLDOUT_MAPE_result.append(Holdout_mape)
        HOLDOUT_MEDAE_result.append(Holdout_medae)
        HOLDOUT_MAXE_result.append(Holdout_maxe)

        # Form new predicted output dataframe by concatenating dataframes
        TRAIN_PRED = pd.concat([TRAIN_PRED, Train_Pred], axis=1)
        HOLDOUT_PRED = pd.concat([HOLDOUT_PRED, Holdout_Pred], axis=1)
        NEW_TEST_PRED_2 = pd.concat([NEW_TEST_PRED_2, New_Test_pred_2], axis=1)

        # Training regression plot
        print('Model: {} - Random_state : {} - R2: {:.5f} | MAE: {:.5f} | MSE: {:.5f} | RMSE: {:.5f} | MAPE: {:.5f} | MEDAE: {:.5f} | MAXE: {:.5f}'.format(reg_n, i,
                                                                                            Train_r2, Train_mae, Train_mse, Train_rmse,
                                                                                            Train_mape, Train_medae, Train_maxe))

        # Holdout regression plot
        print('Model: {} - Random_state : {} - R2: {:.5f} | MAE: {:.5f} | MSE: {:.5f} | RMSE: {:.5f} | MAPE: {:.5f} | MEDAE: {:.5f} | MAXE: {:.5f}'.format(reg_n, i,
                                                                                            Holdout_r2, Holdout_mae, Holdout_mse, Holdout_rmse,
                                                                                            Holdout_mape, Holdout_medae, Holdout_maxe))

    Train_R2 = pd.DataFrame(TRAIN_R2_result, columns=[str(reg_n)])
    TRAIN_R2 = pd.concat([TRAIN_R2, Train_R2],axis=1)

    Train_MAE = pd.DataFrame(TRAIN_MAE_result, columns=[str(reg_n)])
    TRAIN_MAE = pd.concat([TRAIN_MAE, Train_MAE],axis=1)

    Train_MSE = pd.DataFrame(TRAIN_MSE_result, columns=[str(reg_n)])
    TRAIN_MSE = pd.concat([TRAIN_MSE, Train_MSE],axis=1)

    Train_RMSE = pd.DataFrame(TRAIN_RMSE_result, columns=[str(reg_n)])
    TRAIN_RMSE = pd.concat([TRAIN_RMSE, Train_RMSE],axis=1)

    Train_MAPE = pd.DataFrame(TRAIN_MAPE_result, columns=[str(reg_n)])
    TRAIN_MAPE = pd.concat([TRAIN_MAPE, Train_MAPE],axis=1)

    Train_MEDAE = pd.DataFrame(TRAIN_MEDAE_result, columns=[str(reg_n)])
    TRAIN_MEDAE = pd.concat([TRAIN_MEDAE, Train_MEDAE],axis=1)

    Train_MAXE = pd.DataFrame(TRAIN_MAXE_result, columns=[str(reg_n)])
    TRAIN_MAXE = pd.concat([TRAIN_MAXE, Train_MAXE],axis=1)


    Holdout_R2 = pd.DataFrame(HOLDOUT_R2_result, columns=[str(reg_n)])
    HOLDOUT_R2 = pd.concat([HOLDOUT_R2, Holdout_R2],axis=1)

    Holdout_MAE = pd.DataFrame(HOLDOUT_MAE_result, columns=[str(reg_n)])
    HOLDOUT_MAE = pd.concat([HOLDOUT_MAE, Holdout_MAE],axis=1)

    Holdout_MSE = pd.DataFrame(HOLDOUT_MSE_result, columns=[str(reg_n)])
    HOLDOUT_MSE = pd.concat([HOLDOUT_MSE, Holdout_MSE],axis=1)

    Holdout_RMSE = pd.DataFrame(HOLDOUT_RMSE_result, columns=[str(reg_n)])
    HOLDOUT_RMSE = pd.concat([HOLDOUT_RMSE, Holdout_RMSE],axis=1)

    Holdout_MAPE = pd.DataFrame(HOLDOUT_MAPE_result, columns=[str(reg_n)])
    HOLDOUT_MAPE = pd.concat([HOLDOUT_MAPE, Holdout_MAPE],axis=1)

    Holdout_MEDAE = pd.DataFrame(HOLDOUT_MEDAE_result, columns=[str(reg_n)])
    HOLDOUT_MEDAE = pd.concat([HOLDOUT_MEDAE, Holdout_MEDAE],axis=1)

    Holdout_MAXE = pd.DataFrame(HOLDOUT_MAXE_result, columns=[str(reg_n)])
    HOLDOUT_MAXE = pd.concat([HOLDOUT_MAXE, Holdout_MAXE],axis=1)

Thu Jul 17 09:24:59 2025
Model: skorch_model_1 - Random_state : 0 - R2: 0.80689 | MAE: 9.66634 | MSE: 268.65237 | RMSE: 16.39062 | MAPE: 46447684747264.00000 | MEDAE: 6.05038 | MAXE: 294.04282
Model: skorch_model_1 - Random_state : 0 - R2: 0.70718 | MAE: 11.71755 | MSE: 405.03528 | RMSE: 20.12549 | MAPE: 16682429972480.00000 | MEDAE: 7.24439 | MAXE: 290.04282
Thu Jul 17 09:26:27 2025
Model: skorch_model_1 - Random_state : 1 - R2: 0.78299 | MAE: 10.26099 | MSE: 302.99979 | RMSE: 17.40689 | MAPE: 27771428405248.00000 | MEDAE: 6.42065 | MAXE: 333.82434
Model: skorch_model_1 - Random_state : 1 - R2: 0.71351 | MAE: 11.65711 | MSE: 390.31305 | RMSE: 19.75634 | MAPE: 74800856825856.00000 | MEDAE: 7.06495 | MAXE: 335.82434
Thu Jul 17 09:27:56 2025
Model: skorch_model_1 - Random_state : 2 - R2: 0.79958 | MAE: 9.60142 | MSE: 273.46066 | RMSE: 16.53665 | MAPE: 44687847063552.00000 | MEDAE: 5.88986 | MAXE: 319.00485
Model: skorch_model_1 - Random_state : 2 - R2: 0.71890 | MAE: 11.87096 | MSE: 418.

In [ ]:
TRAIN_R2

,skorch_model_1,skorch_model_2,skorch_model_3
0,0.806888,0.808849,0.809647
1,0.782987,0.793952,0.784870
2,0.799581,0.797832,0.796766
3,0.798678,0.801551,0.790291
4,0.800670,0.796256,0.802728
...,...,...,...
95,0.781470,0.782406,0.789901
96,0.798469,0.795476,0.798595
97,0.791986,0.792500,0.796099
98,0.802028,0.806713,0.803992


In [ ]:
TRAIN_R2.to_csv('G_VRH_TRAIN_R2.csv', index=False)
!head G_VRH_TRAIN_R2.csv

skorch_model_1,skorch_model_2,skorch_model_3
0.8068884259299233,0.8088493940533903,0.8096472221517361
0.7829866483157575,0.7939515593680849,0.7848703232824343
0.7995805249630656,0.7978322267968907,0.7967660061878443
0.7986775117249894,0.801550772315621,0.7902908909099331
0.8006695926167002,0.7962560429433587,0.8027279646709721
0.7895227881632183,0.7968771905544212,0.7967882963523957
0.7889595913093088,0.7877234877717512,0.7916382480067228
0.8019821220425098,0.7967999477195339,0.7965770082206278
0.7914537007803357,0.792181852492479,0.8034642777912898


In [ ]:
TRAIN_MAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,9.666342,9.576007,9.676303
1,10.260988,9.893533,10.197639
2,9.601419,9.788536,9.723183
3,9.701451,9.865198,9.767004
4,9.669415,9.774500,9.739907
...,...,...,...
95,10.566640,10.510260,10.248601
96,9.855080,9.955040,9.895191
97,9.846526,9.885170,9.781718
98,9.887255,9.801980,9.854844


In [ ]:
TRAIN_MAE.to_csv('G_VRH_TRAIN_MAE.csv', index=False)
!head G_VRH_TRAIN_MAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
9.666342,9.576007,9.676303
10.260988,9.893533,10.197639
9.601419,9.788536,9.723183
9.701451,9.865198,9.767004
9.669415,9.7745,9.739907
9.844891,9.909281,9.684445
9.967239,10.032862,9.950285
9.885769,10.056781,10.035587
9.879359,10.244411,9.675949


In [ ]:
TRAIN_MSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,268.652374,265.924316,264.814423
1,302.999786,287.690308,300.369781
2,273.460663,275.846130,277.300934
3,273.300415,269.399902,284.685455
4,271.479614,277.490662,268.676178
...,...,...,...
95,300.529419,299.242279,288.934479
96,286.175964,290.425568,285.996979
97,294.545441,293.817841,288.721222
98,267.522675,261.191040,264.868591


In [ ]:
TRAIN_MSE.to_csv('G_VRH_TRAIN_MSE.csv', index=False)
!head G_VRH_TRAIN_MSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
268.65237,265.92432,264.81442
302.9998,287.6903,300.36978
273.46066,275.84613,277.30093
273.3004,269.3999,284.68546
271.4796,277.49066,268.67618
289.7286,279.605,279.72742
292.1866,293.898,288.478
270.1493,277.21915,277.5233
292.57727,291.5557,275.72717


In [ ]:
TRAIN_RMSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,16.390619,16.307186,16.273119
1,17.406889,16.961435,17.331179
2,16.536646,16.608616,16.652355
3,16.531799,16.413406,16.872625
4,16.476639,16.658051,16.391344
...,...,...,...
95,17.335785,17.298620,16.998074
96,16.916737,17.041878,16.911446
97,17.162327,17.141115,16.991798
98,16.356121,16.161406,16.274784


In [ ]:
TRAIN_RMSE.to_csv('G_VRH_TRAIN_RMSE.csv', index=False)
!head G_VRH_TRAIN_RMSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
16.39062,16.307186,16.273119
17.406889,16.961435,17.331179
16.536646,16.608616,16.652355
16.5318,16.413406,16.872625
16.476639,16.65805,16.391344
17.021416,16.721394,16.725054
17.093468,17.143454,16.98464
16.436218,16.649899,16.65903
17.10489,17.075003,16.605034


In [ ]:
TRAIN_MAPE

,skorch_model_1,skorch_model_2,skorch_model_3
0,4.644768e+13,4.520414e+13,4.862616e+13
1,2.777143e+13,2.701901e+13,2.543649e+13
2,4.468785e+13,4.467125e+13,4.726260e+13
3,5.527130e+13,5.348122e+13,6.102777e+13
4,3.606845e+13,3.647194e+13,3.515265e+13
...,...,...,...
95,5.000772e+13,4.938171e+13,5.473136e+13
96,5.888685e+13,7.674781e+13,6.216191e+13
97,4.316724e+13,4.377263e+13,4.149315e+13
98,3.564269e+13,3.699259e+13,3.926140e+13


In [ ]:
TRAIN_MAPE.to_csv('G_VRH_TRAIN_MAPE.csv', index=False)
!head G_VRH_TRAIN_MAPE.csv

skorch_model_1,skorch_model_2,skorch_model_3
46447685000000.0,45204140000000.0,48626156000000.0
27771428000000.0,27019008000000.0,25436493000000.0
44687847000000.0,44671246000000.0,47262604000000.0
55271300000000.0,53481220000000.0,61027770000000.0
36068447000000.0,36471944000000.0,35152646000000.0
23766986000000.0,27127112000000.0,23858598000000.0
44402680000000.0,44623444000000.0,45890014000000.0
33619860000000.0,35565303000000.0,36500890000000.0
37768068000000.0,34329235000000.0,38164600000000.0


In [ ]:
TRAIN_MEDAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,6.050381,6.003796,6.144478
1,6.420647,6.182165,6.375102
2,5.889858,6.183912,6.082762
3,6.005059,6.269618,6.079682
4,6.161125,6.246960,6.239035
...,...,...,...
95,6.797810,6.807008,6.572251
96,6.140723,6.224274,6.184087
97,6.060606,6.043179,6.018660
98,6.327521,6.339046,6.349693


In [ ]:
TRAIN_MEDAE.to_csv('G_VRH_TRAIN_MEDAE.csv', index=False)
!head G_VRH_TRAIN_MEDAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
6.0503807,6.0037956,6.144478
6.4206467,6.182165,6.375102
5.8898582,6.1839123,6.082762
6.0050592,6.269618,6.0796824
6.161125,6.24696,6.2390347
6.234359,6.285327,6.1248455
6.2146792,6.2358646,6.140994
6.491194,6.5318165,6.444073
6.15854,6.526207,6.114208


In [ ]:
TRAIN_MAXE

,skorch_model_1,skorch_model_2,skorch_model_3
0,294.042816,292.242096,276.348175
1,333.824341,325.314148,339.555389
2,319.004852,319.278107,290.404114
3,227.439362,293.779602,242.565399
4,228.120972,242.126068,230.243622
...,...,...,...
95,347.508728,339.498596,340.588837
96,268.546600,245.813965,259.322906
97,307.366577,296.203369,304.306580
98,366.097931,353.243652,335.696533


In [ ]:
TRAIN_MAXE.to_csv('G_VRH_TRAIN_MAXE.csv', index=False)
!head G_VRH_TRAIN_MAXE.csv

skorch_model_1,skorch_model_2,skorch_model_3
294.04282,292.2421,276.34818
333.82434,325.31415,339.5554
319.00485,319.2781,290.4041
227.43936,293.7796,242.5654
228.12097,242.12607,230.24362
366.4256,316.1325,344.3379
304.4793,311.72235,299.95047
288.1803,273.7193,295.10806
356.26,301.18024,330.10623


In [ ]:
HOLDOUT_R2

,skorch_model_1,skorch_model_2,skorch_model_3
0,0.707177,0.704903,0.704180
1,0.713509,0.717749,0.710159
2,0.718905,0.715654,0.724700
3,0.742376,0.729956,0.745453
4,0.733397,0.733421,0.731120
...,...,...,...
95,0.684352,0.689212,0.691374
96,0.727996,0.723302,0.724519
97,0.740848,0.749215,0.749621
98,0.654525,0.662655,0.662091


In [ ]:
HOLDOUT_R2.to_csv('G_VRH_HOLDOUT_R2.csv', index=False)
!head G_VRH_HOLDOUT_R2.csv

skorch_model_1,skorch_model_2,skorch_model_3
0.707177105075115,0.7049028299930582,0.7041797693707574
0.7135089784267352,0.7177494827570001,0.7101591996523584
0.7189045978497672,0.7156537355120143,0.7246997257117473
0.7423760089841915,0.7299558281950956,0.7454525386581854
0.7333966144573076,0.7334212134548861,0.7311196848638832
0.6829857209162722,0.6984096077747088,0.6900605714752835
0.7164275819509358,0.7177949719240407,0.7154931628684795
0.7348987934872726,0.7295321576786951,0.7300472867026084
0.7192977555251481,0.7071793398595688,0.7229812743986148


In [ ]:
HOLDOUT_MAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,11.717547,11.749828,11.764357
1,11.657111,11.441254,11.727391
2,11.870962,12.034084,11.899900
3,11.787101,11.868692,11.746975
4,11.511477,11.603089,11.595965
...,...,...,...
95,12.301715,12.171711,11.950183
96,11.496893,11.697932,11.615949
97,11.290599,11.290648,11.212033
98,12.276257,12.198895,12.245809


In [ ]:
HOLDOUT_MAE.to_csv('G_VRH_HOLDOUT_MAE.csv', index=False)
!head G_VRH_HOLDOUT_MAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
11.717547,11.749828,11.764357
11.657111,11.441254,11.727391
11.870962,12.034084,11.8999
11.787101,11.868692,11.746975
11.5114765,11.603089,11.595965
12.183607,12.144987,12.148906
11.844328,11.832047,11.822162
11.674395,11.942415,11.733245
11.358232,11.732749,11.264646


In [ ]:
HOLDOUT_MSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,405.035278,408.181122,409.181244
1,390.313049,384.535828,394.876740
2,418.851318,423.695343,410.216217
3,391.013702,409.864655,386.344208
4,399.919342,399.882446,403.334869
...,...,...,...
95,456.629547,449.598846,446.470734
96,344.803009,350.753265,349.210846
97,332.679504,321.937683,321.416473
98,532.935242,520.394104,521.263855


In [ ]:
HOLDOUT_MSE.to_csv('G_VRH_HOLDOUT_MSE.csv', index=False)
!head G_VRH_HOLDOUT_MSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
405.03528,408.18112,409.18124
390.31305,384.53583,394.87674
418.85132,423.69534,410.21622
391.0137,409.86465,386.3442
399.91934,399.88245,403.33487
456.83212,434.60556,446.63693
399.80286,397.875,401.12027
395.1145,403.11307,402.3453
375.03424,391.2251,370.11285


In [ ]:
HOLDOUT_RMSE

,skorch_model_1,skorch_model_2,skorch_model_3
0,20.125488,20.203493,20.228230
1,19.756342,19.609585,19.871506
2,20.465857,20.583860,20.253796
3,19.774067,20.245115,19.655642
4,19.997984,19.997061,20.083199
...,...,...,...
95,21.368893,21.203747,21.129854
96,18.568872,18.728409,18.687183
97,18.239504,17.942621,17.928091
98,23.085390,22.812149,22.831203


In [ ]:
HOLDOUT_RMSE.to_csv('G_VRH_HOLDOUT_RMSE.csv', index=False)
!head G_VRH_HOLDOUT_RMSE.csv

skorch_model_1,skorch_model_2,skorch_model_3
20.125488,20.203493,20.22823
19.756342,19.609585,19.871506
20.465857,20.58386,20.253796
19.774067,20.245115,19.655642
19.997984,19.99706,20.083199
21.373632,20.847195,21.133787
19.995071,19.946804,20.027987
19.877487,20.077677,20.058546
19.3658,19.779411,19.238317


In [ ]:
HOLDOUT_MAPE

,skorch_model_1,skorch_model_2,skorch_model_3
0,1.668243e+13,1.667316e+13,1.293525e+13
1,7.480086e+13,7.071098e+13,7.649961e+13
2,7.971230e+12,9.521551e+12,9.198835e+12
3,2.832256e+13,3.292095e+13,2.994005e+13
4,2.034554e+13,2.513618e+13,3.148053e+13
...,...,...,...
95,3.786917e+13,2.861227e+13,3.653910e+13
96,1.993099e+13,2.702590e+13,1.704166e+13
97,6.881257e+13,6.442404e+13,6.378216e+13
98,4.258047e+13,4.934374e+13,4.171686e+13


In [ ]:
HOLDOUT_MAPE.to_csv('G_VRH_HOLDOUT_MAPE.csv', index=False)
!head G_VRH_HOLDOUT_MAPE.csv

skorch_model_1,skorch_model_2,skorch_model_3
16682430000000.0,16673156000000.0,12935247000000.0
74800860000000.0,70710980000000.0,76499610000000.0
7971230000000.0,9521551000000.0,9198835000000.0
28322562000000.0,32920950000000.0,29940053000000.0
20345537000000.0,25136180000000.0,31480531000000.0
109290620000000.0,119411675000000.0,119777330000000.0
10294163000000.0,7784161600000.0,11492442000000.0
81373314000000.0,106837350000000.0,93017835000000.0
49068880000000.0,53539270000000.0,49176948000000.0


In [ ]:
HOLDOUT_MEDAE

,skorch_model_1,skorch_model_2,skorch_model_3
0,7.244387,7.084267,7.223545
1,7.064947,6.855328,7.312537
2,7.177822,7.327061,7.304008
3,7.214705,7.225632,7.158939
4,6.890450,6.897917,6.980404
...,...,...,...
95,7.780497,7.504868,7.174417
96,6.996510,7.308491,7.225006
97,7.035347,7.080589,7.057114
98,7.211526,7.238386,7.220381


In [ ]:
HOLDOUT_MEDAE.to_csv('G_VRH_HOLDOUT_MEDAE.csv', index=False)
!head G_VRH_HOLDOUT_MEDAE.csv

skorch_model_1,skorch_model_2,skorch_model_3
7.2443867,7.0842667,7.223545
7.064947,6.8553276,7.312537
7.177822,7.3270607,7.3040085
7.2147045,7.2256317,7.1589394
6.8904495,6.897917,6.980404
7.056246,7.068924,7.169182
7.030422,7.080658,7.0139084
7.329342,7.5238037,7.318675
6.869442,7.3898315,6.9822655


In [ ]:
HOLDOUT_MAXE

,skorch_model_1,skorch_model_2,skorch_model_3
0,290.042816,288.242096,272.348175
1,335.824341,327.314148,341.555389
2,321.004852,321.278107,292.404114
3,229.439362,295.779602,251.565399
4,240.385330,251.126068,240.588348
...,...,...,...
95,345.508728,337.498596,338.588837
96,208.551941,202.430328,210.538391
97,255.512970,231.082611,243.585464
98,370.097931,357.243652,339.696533


In [ ]:
HOLDOUT_MAXE.to_csv('G_VRH_HOLDOUT_MAXE.csv', index=False)
!head G_VRH_HOLDOUT_MAXE.csv

skorch_model_1,skorch_model_2,skorch_model_3
290.04282,288.2421,272.34818
335.82434,327.31415,341.5554
321.00485,321.2781,292.4041
229.43936,295.7796,251.5654
240.38533,251.12607,240.58835
364.4256,314.1325,342.3379
306.4793,313.72235,301.95047
290.1803,275.7193,297.10806
354.26,299.18024,328.10623


In [ ]:
NEW_TEST_PRED_2

,skorch_model_1_0,skorch_model_1_1,skorch_model_1_2,skorch_model_1_3,skorch_model_1_4,skorch_model_1_5,skorch_model_1_6,skorch_model_1_7,skorch_model_1_8,skorch_model_1_9,...,skorch_model_3_90,skorch_model_3_91,skorch_model_3_92,skorch_model_3_93,skorch_model_3_94,skorch_model_3_95,skorch_model_3_96,skorch_model_3_97,skorch_model_3_98,skorch_model_3_99
0,16.963572,18.786228,18.990244,22.694754,19.291109,18.875664,18.027206,17.880203,18.739746,20.774302,...,16.345879,20.068865,13.812319,25.433441,21.349005,11.971652,24.493422,19.985355,12.490821,22.285744
1,50.740250,53.793007,54.281281,46.846878,53.063599,45.336578,48.170887,46.041439,51.635311,54.010513,...,47.399220,51.134106,50.038818,44.315022,54.972107,59.372868,45.123302,47.002983,54.001900,56.336178
2,46.958466,49.280865,50.419662,49.258160,44.622353,45.455925,45.111599,44.372074,47.447632,51.009949,...,49.118572,44.441460,49.063972,41.779385,43.677422,51.585262,43.461163,47.830067,43.694023,43.039036
3,25.400103,22.856728,17.596827,20.235781,19.288137,17.901833,19.551781,20.378044,20.388075,22.801731,...,19.634722,19.919847,22.327787,20.995708,18.873415,21.161648,19.001266,21.258499,20.085516,20.643345
4,49.086288,47.804832,42.638805,41.715851,42.271324,39.665760,42.353352,47.638775,47.140278,47.738693,...,45.028996,40.218349,52.040359,43.694530,42.819603,49.262661,44.996391,46.706085,44.912209,50.254448
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4738,41.487526,33.385780,23.742651,38.627144,26.866381,46.659340,46.593124,34.533386,41.534382,36.718464,...,56.360252,28.673515,33.613144,45.762379,42.651764,52.072350,26.582664,42.507229,19.838739,40.438942
4739,40.889809,28.353071,26.927340,32.575455,26.693922,46.761673,42.223934,28.706776,40.786663,35.574707,...,54.426414,29.289917,28.115978,43.265972,43.959198,47.153255,28.428701,41.003109,20.035627,41.775616
4740,42.535797,34.268360,19.365576,38.690086,24.306154,44.999912,47.320129,34.153011,36.904671,37.819183,...,54.008976,30.561710,28.495611,44.496700,44.139107,48.283535,28.217947,40.418247,22.076353,40.570351
4741,52.408794,42.374912,20.774038,57.596169,30.200548,51.361263,55.349060,46.998062,42.289513,47.520729,...,61.017433,46.047577,41.442665,55.111557,55.278549,43.318501,35.203819,52.762489,27.034712,46.638668


In [ ]:
NEW_TEST_PRED_2.to_csv('NEW-G_VRH_predictions_for_SCREENED_Li-based_Na_based_K-based_Mg-based_Ba-based_Ca-based_and_Al-based_compounds_from_MPJ_with_no_mech_prop_info.csv', index=False)
!head NEW-G_VRH_predictions_for_SCREENED_Li-based_Na_based_K-based_Mg-based_Ba-based_Ca-based_and_Al-based_compounds_from_MPJ_with_no_mech_prop_info.csv

skorch_model_1_0,skorch_model_1_1,skorch_model_1_2,skorch_model_1_3,skorch_model_1_4,skorch_model_1_5,skorch_model_1_6,skorch_model_1_7,skorch_model_1_8,skorch_model_1_9,skorch_model_1_10,skorch_model_1_11,skorch_model_1_12,skorch_model_1_13,skorch_model_1_14,skorch_model_1_15,skorch_model_1_16,skorch_model_1_17,skorch_model_1_18,skorch_model_1_19,skorch_model_1_20,skorch_model_1_21,skorch_model_1_22,skorch_model_1_23,skorch_model_1_24,skorch_model_1_25,skorch_model_1_26,skorch_model_1_27,skorch_model_1_28,skorch_model_1_29,skorch_model_1_30,skorch_model_1_31,skorch_model_1_32,skorch_model_1_33,skorch_model_1_34,skorch_model_1_35,skorch_model_1_36,skorch_model_1_37,skorch_model_1_38,skorch_model_1_39,skorch_model_1_40,skorch_model_1_41,skorch_model_1_42,skorch_model_1_43,skorch_model_1_44,skorch_model_1_45,skorch_model_1_46,skorch_model_1_47,skorch_model_1_48,skorch_model_1_49,skorch_model_1_50,skorch_model_1_51,skorch_model_1_52,skorch_model_1_53,skorch_model_1_54,skorch_model_1_55,sk

In [ ]:
TRAIN_R2.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,0.796372,0.795554,0.796559
std,0.010908,0.011857,0.011028
min,0.765404,0.762405,0.767426
25%,0.789516,0.788909,0.791908
50%,0.796164,0.796443,0.797479
75%,0.804796,0.802808,0.803858
max,0.823793,0.820222,0.821132


In [ ]:
HOLDOUT_R2.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,0.711665,0.711069,0.712149
std,0.021069,0.021181,0.020991
min,0.654525,0.656144,0.662091
25%,0.702189,0.698165,0.698570
50%,0.713484,0.713321,0.711297
75%,0.726111,0.726987,0.725881
max,0.751928,0.755483,0.756263


In [ ]:
TRAIN_MAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,9.862313,9.895566,9.878449
std,0.234972,0.245860,0.206152
min,9.298800,9.372348,9.368382
25%,9.711415,9.766103,9.736262
50%,9.845708,9.859699,9.864133
75%,9.989007,10.035567,10.011978
max,10.663296,10.682771,10.479882


In [ ]:
HOLDOUT_MAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,11.755284,11.772787,11.754841
std,0.293934,0.303457,0.300283
min,10.899699,11.017240,11.017110
25%,11.545909,11.590847,11.589120
50%,11.743907,11.761097,11.747270
75%,11.913507,11.948392,11.925633
max,12.497178,12.663488,12.546870


In [ ]:
TRAIN_MSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,282.435760,283.561829,282.163727
std,16.714596,17.790030,16.664938
min,235.295898,240.063736,238.849564
25%,271.370895,274.121513,272.148422
50%,281.119293,281.235245,280.717163
75%,292.888359,291.740730,289.151848
max,321.319641,334.452118,332.132660


In [ ]:
HOLDOUT_MSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,404.000092,404.702972,403.260742
std,42.181545,40.819969,41.414955
min,322.717133,321.937683,320.239685
25%,376.217857,375.120262,372.957382
50%,401.341919,403.048019,403.140396
75%,425.224571,428.940521,428.094490
max,532.935242,521.695435,521.263855


In [ ]:
TRAIN_RMSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,16.798519,16.831146,16.790529
std,0.498024,0.526368,0.494323
min,15.339357,15.493990,15.454759
25%,16.473338,16.556612,16.496922
50%,16.766611,16.770069,16.754616
75%,17.113975,17.080418,17.004463
max,17.925390,18.288033,18.224506


In [ ]:
HOLDOUT_RMSE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,20.073275,20.092295,20.055548
std,1.036569,1.006335,1.022790
min,17.964329,17.942621,17.895243
25%,19.396328,19.368013,19.312095
50%,20.033519,20.076056,20.078356
75%,20.620970,20.710873,20.690445
max,23.085390,22.840652,22.831203


In [ ]:
TRAIN_MAPE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,1.000000e+02,1.000000e+02,1.000000e+02
mean,4.454013e+13,4.475940e+13,4.485931e+13
std,1.487775e+13,1.670944e+13,1.643097e+13
min,1.279505e+13,1.136117e+13,1.564728e+13
25%,3.556945e+13,3.661483e+13,3.647330e+13
50%,4.215761e+13,4.273911e+13,4.268234e+13
75%,5.279858e+13,5.236893e+13,5.206014e+13
max,9.990168e+13,1.202101e+14,1.439069e+14


In [ ]:
HOLDOUT_MAPE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,1.000000e+02,1.000000e+02,1.000000e+02
mean,5.616952e+13,5.940417e+13,5.816488e+13
std,5.051995e+13,5.357174e+13,5.462645e+13
min,1.080052e+12,1.309894e+12,1.402008e+12
25%,1.239892e+13,1.401696e+13,1.223756e+13
50%,3.849335e+13,4.472270e+13,4.412789e+13
75%,9.159095e+13,9.765138e+13,9.378470e+13
max,1.932373e+14,2.082759e+14,2.581513e+14


In [ ]:
TRAIN_MEDAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,6.211908,6.233015,6.221871
std,0.170958,0.187372,0.150215
min,5.875614,5.917804,5.938169
25%,6.090764,6.098605,6.105474
50%,6.195644,6.197443,6.214440
75%,6.308053,6.313176,6.320999
max,6.797810,6.879985,6.572251


In [ ]:
HOLDOUT_MEDAE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,7.139203,7.161275,7.156289
std,0.209888,0.217784,0.187665
min,6.752022,6.630013,6.702785
25%,7.000764,7.026534,7.018323
50%,7.093252,7.130659,7.141873
75%,7.259710,7.322905,7.298604
max,7.799710,7.803799,7.697714


In [ ]:
TRAIN_MAXE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,294.903625,292.794189,290.482880
std,43.939873,44.394394,43.567196
min,224.238525,227.151184,222.219910
25%,259.075539,252.937263,259.843079
50%,286.732315,291.954697,281.062363
75%,328.869637,322.719437,319.665894
max,430.217285,423.144958,415.880524


In [ ]:
HOLDOUT_MAXE.describe()

,skorch_model_1,skorch_model_2,skorch_model_3
count,100.000000,100.000000,100.000000
mean,283.607788,281.705383,278.744415
std,51.018173,50.196026,49.622566
min,151.187744,150.633331,146.522095
25%,241.193020,244.181755,243.209824
50%,277.873917,273.279678,273.525986
75%,326.404907,314.730247,310.871574
max,427.217285,420.144958,413.880524
